# Observation planning TEMPLATE

## Using astroplan to find out which nights an object is up

This notebook focuses on two broader questions:

1. Which times of year is it possible to observe the object of interest at a particular location?
2. On a particular date, which times of night is it best to observe the object of interest?

The python package [astroplan](https://astroplan.readthedocs.io) will be used to do all of the work here. As a result, much of this notebook will focus on the mechanics of how to do that rather than on background or theory.


## Exoplanet resources for epoch, period and more

+ The [Exoplanet Transit Database (ETD)](http://var2.astro.cz/ETD/index.php) is really handy for many exoplanets discovered from ground-baed observatories.
+ For TESS planet candidates (those whose name starts `TIC`) use [ExoFOP-TESS](https://exofop.ipac.caltech.edu/tess/) and put the "TIC number" into the appropriate search box. The epoch and period will be included in the page of information that comes up.
+ The most complete database is the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/).

## Variable star resources for period, epoch and more

+ The best one by far is the [AAVSO Variable Star Index (VSX)](https://www.aavso.org/vsx/index.php?view=search.top). In addition to period and epoch it can also generate an ephemeris of upcoming events, i.e. a list of upcoming maxima or eclipses. 

## Run, but do not modify, the code in the cell below

None of the following cells will run until you have run the one below.

In [ ]:
# Set plotting interface
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import dates
import numpy as np

# Load a couple of things needed from astroplan
from astroplan import (Observer, FixedTarget, 
                       AtNightConstraint, AltitudeConstraint,
                       is_event_observable)
from astroplan.plots import plot_airmass, plot_finder_image

# And load Time from astropy
from astropy.time import Time

# Load astropy units module
import astropy.units as u

# Get the latest Earth orientation information...

from astroplan import download_IERS_A
#download_IERS_A()

Visibility depends on location, so the first step is to provide the location of the observatory. Notice that the longitude is given as degrees *east*; it should be a number between 0 and 360.

In [ ]:
feder = Observer(latitude="46.86678d", longitude="263.54672d")

Everything done below can be doen for either a single object, as below, or for several objects at once. As a first example we will stick to one object, but an example with a couple of objects will be included at the end.

In [ ]:
my_object = FixedTarget.from_name('ey uma')

## Plotting visibility over one night

Here, the goal is to plot, over the course of an entire day, how high EY UMa is above the horizon.

Recall that the first step in providing a time is to convert it from local time to UTC. Central Daylight Time is five hours behind UTC, so to get to UTC, add five hours to 11PM CDT on Thu, Sep **15**, 2016.

Doing so gives 4AM on **Fri**, Sep **16**, 2016 UTC. This time is entered below in ISO format, though astropy does a pretty good job at interpreting sseveral formats.

In [ ]:
obs_time = Time("2016-09-16 04:00:00", scale='utc')

In [ ]:
plot_airmass(my_object, feder, obs_time, brightness_shading=True, altitude_yaxis=True)
plt.title(my_object.name)

## Plot airmass over a large range of dates to make a rough choice of night

The cell immediately below this generates a list of dates, 5 days apart from each other, covering 365 days.

In [ ]:
times = obs_time + np.arange(0, 365, 5) * u.day

In [ ]:
ax = plot_airmass(my_object, feder, times, altitude_yaxis=True)
plt.title('target visibility at 10PM or 11PM local time'.format(my_object.name))

plt.grid()

# Code below adds date to the horizontal axis
date_formatter = dates.DateFormatter('%D %H:%M')
ax.xaxis.set_major_formatter(date_formatter)


## Revising the plan

From the graph above it is clear that an obsevrvation date between roughly the beginning of December and late May would work better than September.

The plot below shows visibility over the night of Dec 1, 2016 at 11:59 CST. That is Dec **2**, 2016 at 06:00 UTC because the offset between Central *Standard* Time and UTC is 6 hours.

In [ ]:
obs_time = Time("2020-12-02 05:59:00", scale='utc')

In [ ]:
ax = plot_airmass(my_object, feder, obs_time, brightness_shading=False, altitude_yaxis=True)
plt.grid()
plt.title(my_object.name)

It appears EY UMa will be observable from roughly 4AM UTC (10PM CST on 12/1) until the sky begins to brighten at 12:00UTC (6AM CST)

### Predicting upcoming maximum or eclipse

The formula for predicting maxima given an epoch, called $t_{\text{epoch}}$ below, and a period, called $P$ below, is

\begin{equation}
t_{\text{future}} = t_{\text{epoch}} + N * P
\end{equation}

Let's use that to calculate some of the future maxima of EY Uma

In [ ]:
t_epoch = Time(2456159.32, scale='utc', format='jd')
period = 0.54909 * u.day

# You will need to change this value to something like
# the time now minus the epoch divided by the period.
now = Time('2020-10-06T13:59', scale='utc')
n_start = (now - t_epoch).jd // period.value 

n_end = n_start + 60

future_events = t_epoch + period * np.arange(n_start, n_end)

In [ ]:
for event in future_events:
    print(event.isot)

In [ ]:
constraints = [AtNightConstraint.twilight_civil(),
               AltitudeConstraint(min=40*u.deg)]
is_observable = is_event_observable(constraints, feder, my_object, 
                                    times=future_events)
is_observable = is_observable[0]

In [ ]:
for event in future_events[is_observable]:
    print(event.isot)

## Creating a finding chart

A finding chart is simply a picture (or diagram) of the region of the sky surrounding the object of interest. The code below generates a chart for `my_object`. You can change the options in `plot_finder_image` to turn off the reticle, which marks the object of interest, or to turn off the grid.

In [ ]:
plt.figure(figsize=(10, 10))
ax, hdu = plot_finder_image(my_object, fov_radius=20 * u.arcmin, reticle=True, grid=True)